# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [2]:
import json
import os

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
import autogen
autogen.__version__

'0.9.1post0'

# Example: Group chat for an online shopping problem

- ## Product Finder

- ## Purchase Advisor

- ## Checkout Assistant

- ## Customer

## Creating the Necessary Agents

In [11]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# Product finder agent
finder_message = """You are a smart product finder agent.
Given a general need or interest, suggest a suitable product someone might want to buy online. Mention the product name and a one-line reason why it's a good fit.
"""

# Purchase advisor agent
advisor_message = """You are a purchase advisor agent.
Given a product name, generate pros and cons, price estimate, and suggest one alternative product with reasoning.
"""

# Checkout assistant agent
checkout_message = """You are a checkout assistant agent.
Given a chosen product, provide a brief checkout summary including product, estimated price, shipping method, and a friendly thank-you message.
"""

# Customer initiates the shopping process
customer_message = """You are an online shopper.
You describe your needs, review purchase advice (with one round of follow-up), and then confirm a product to buy.
"""

with llm_config:
    product_finder = ConversableAgent(
        name="finder_agent",
        system_message=finder_message,
    )

    purchase_advisor = ConversableAgent(
        name="advisor_agent",
        system_message=advisor_message,
    )

    checkout_assistant = ConversableAgent(
        name="checkout_agent",
        system_message=checkout_message,
    )

    customer = ConversableAgent(
        name="customer_agent",
        system_message=customer_message,
    )


# Create the pattern

In [17]:
pattern = AutoPattern(
    initial_agent=product_finder,         # Start with the customer
    agents=[product_finder, purchase_advisor, checkout_assistant],           # All agents in the group chat
    user_agent=customer,                            # Provide our human-in-the-loop agent
    group_manager_args={"llm_config": llm_config}  # Config for group manager
)

# Initialize the group chat

In [19]:
initial_prompt = """I'm looking for a Laptop suitable for training deep learning models."""

result, context_variables, last_agent = initiate_group_chat(
    pattern=pattern,
    messages=initial_prompt,                     # Initial request with transactions
    max_rounds=10,
)

customer_agent (to chat_manager):

I'm looking for a Laptop suitable for training deep learning models.

--------------------------------------------------------------------------------

Next speaker: finder_agent


>>>>>>>> USING AUTO REPLY...
finder_agent (to chat_manager):

**ASUS ROG Zephyrus G14** - This laptop features a powerful AMD Ryzen 9 processor and NVIDIA GeForce RTX 3060 GPU, making it ideal for intensive deep learning tasks.

--------------------------------------------------------------------------------

Next speaker: advisor_agent


>>>>>>>> USING AUTO REPLY...
advisor_agent (to chat_manager):

**ASUS ROG Zephyrus G14**

**Pros:**
1. **Powerful CPU & GPU**: The AMD Ryzen 9 combined with the NVIDIA GeForce RTX 3060 provides excellent performance for deep learning tasks, including model training and data processing.
2. **Portability**: With its compact size and lightweight design, it's suitable for on-the-go professionals or students.
3. **High-Quality Display**: The la